In [16]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import tokenize
import numpy as np
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from lexical_diversity import lex_div as ld
from nltk.util import ngrams
import gensim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import os
import codecs
import re
import string
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from collections import Counter
from collections import OrderedDict
# import nlp
import spacy
from textstat.textstat import textstatistics, legacy_round
from sklearn.preprocessing import StandardScaler
#Using Min Max scaler for NAive bayes so as to bound the values between 0 and 1
#Negative values won't work for probability computations
from sklearn.preprocessing import MinMaxScaler
#GridSearch for hyperparams tuning and classification_report for generating report after param tuning
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

import en_core_web_sm
from sklearn import svm
#Naive Bayes from SKLearn
from sklearn.naive_bayes import MultinomialNB
nlp = en_core_web_sm.load()


In [18]:
data = pd.read_csv('final_features_without_null_values.csv',encoding="ISO-8859-1")

# Split Train and Test Data
X  = data.iloc[:,:-1]
X_train, X_test, y_train, y_test = train_test_split(X, data['class'].values, stratify = data['class'].values, test_size = 0.2)
df_y_train = pd.DataFrame(y_train, columns=['class'])
df_y_test = pd.DataFrame(y_test, columns=['class'])
print("Number of samples in training data :", X_train.shape[0])
print("Number of samples in test data :", X_test.shape[0])

Number of samples in training data : 795
Number of samples in test data : 199


In [19]:
#Check the distribution of data for different classes
train_class_distribution = df_y_train['class'].value_counts().sort_index()
test_class_distribution = df_y_test['class'].value_counts().sort_index()

print("\ntraining distribution:\n\n", train_class_distribution)
print("\ntest distribution: \n\n", test_class_distribution)


training distribution:

 0      2
1      4
2     89
3      5
4      5
5    633
6     14
7     29
8     14
Name: class, dtype: int64

test distribution: 

 1      1
2     22
3      1
4      1
5    159
6      4
7      7
8      4
Name: class, dtype: int64


In [20]:
#Scaling the features using min max normalization as naive bayes cannot handle negative values
scaler = MinMaxScaler()
train_df = scaler.fit_transform(X_train)
test_df = scaler.fit_transform(X_test)

In [21]:
# Alpha=Smoothening parameter(0 for no smoothening)
clf = MultinomialNB(alpha=.45)
clf.fit(train_df, y_train)
pred = clf.predict(test_df)


print('****************Evaluation Metrics (average = macro) for test data original***************')
print('F1 Score: ',metrics.f1_score(y_test, pred, average='macro'))
print('Precision Score: ', metrics.precision_score(y_test, pred, average='macro'))
print('Recall Score: ', metrics.recall_score(y_test, pred, average='macro'))
print('Accuracy: ', metrics.accuracy_score(y_test, pred))


****************Evaluation Metrics (average = macro) for test data original***************
F1 Score:  0.11103351955307263
Precision Score:  0.09987437185929648
Recall Score:  0.125
Accuracy:  0.7989949748743719


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
#Tune hyper parameter using gridsearch cv to set the best hyper parameter
# Set the parameters by cross-validation
tuned_parameters = [{'alpha': [0.45,0.25,0.40,0.55]}]
scores = [ 'recall','f1']

for score in scores:
	print("# Tuning hyper-parameters for %s" % score)
	clf = GridSearchCV(MultinomialNB(), tuned_parameters, scoring='%s_macro' % score)
	clf.fit(train_df, y_train)

	print("Best parameters set found on development set:")
	print(clf.best_params_)
	print("Grid scores on development set:")
	means = clf.cv_results_['mean_test_score']
	stds = clf.cv_results_['std_test_score']
	for mean, std, params in zip(means, stds, clf.cv_results_['params']):
		print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
	print("Detailed classification report:")
	print("The model is trained on the full development set.")
	print("The scores are computed on the full evaluation set.")
	y_true, y_pred = y_test, clf.predict(test_df)


# Tuning hyper-parameters for recall
Best parameters set found on development set:
{'alpha': 0.45}
Grid scores on development set:
0.122 (+/-0.011) for {'alpha': 0.45}
0.122 (+/-0.011) for {'alpha': 0.25}
0.122 (+/-0.011) for {'alpha': 0.4}
0.122 (+/-0.011) for {'alpha': 0.55}
Detailed classification report:
The model is trained on the full development set.
The scores are computed on the full evaluation set.
# Tuning hyper-parameters for f1
Best parameters set found on development set:
{'alpha': 0.45}
Grid scores on development set:
0.108 (+/-0.010) for {'alpha': 0.45}
0.108 (+/-0.010) for {'alpha': 0.25}
0.108 (+/-0.010) for {'alpha': 0.4}
0.108 (+/-0.010) for {'alpha': 0.55}
Detailed classification report:
The model is trained on the full development set.
The scores are computed on the full evaluation set.


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


In [22]:
# Alpha=Smoothening parameter(0 for no smoothening)
# Other hyperparams for naive bayes fit_prior & class prior
# As we don't know prior probability of classes we are going with uniform prior for classes

print('****************Evaluation Metrics (average = macro) for test data original***************')
print('F1 Score: ',metrics.f1_score(y_test, pred, average='macro'))
print('Precision Score: ', metrics.precision_score(y_test, pred, average='macro'))
print('Recall Score: ', metrics.recall_score(y_test, pred, average='macro'))
print('Accuracy: ', metrics.accuracy_score(y_test, pred))

****************Evaluation Metrics (average = macro) for test data original***************
F1 Score:  0.11103351955307263
Precision Score:  0.09987437185929648
Recall Score:  0.125
Accuracy:  0.7989949748743719


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
print('*******************Evaluation Metrics (average = weighted) for test data original*****************************')
print('F1 Score: ',metrics.f1_score(y_test, pred, average='weighted'))
print('Precision Score: ', metrics.precision_score(y_test, pred, average='weighted'))
print('Recall Score: ', metrics.recall_score(y_test, pred, average='weighted'))
print('Accuracy: ', metrics.accuracy_score(y_test, pred))

*******************Evaluation Metrics (average = weighted) for test data original*****************************
F1 Score:  0.7097217933241626
Precision Score:  0.6383929698744981
Recall Score:  0.7989949748743719
Accuracy:  0.7989949748743719


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
